<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao35_API_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Exemplo de Uso da API do Gemini no Google Colab

Este notebook demonstra como começar a usar a API do Google Gemini (através da biblioteca `google-generativeai`) para:
1.  Instalar a biblioteca.
2.  Configurar sua API Key de forma segura.
3.  Gerar texto a partir de um prompt simples.
4.  Manter uma conversa (chat) com histórico.
5.  Usar a capacidade multimodal (enviar texto + imagem).

## Passo 1: Instalação

Primeiro, precisamos instalar a biblioteca Python do Google.

In [1]:
!pip install -q google-generativeai

## Passo 2: Configuração da API Key

Para usar a API, você precisa de uma API Key.

1.  Acesse o [Google AI Studio](https://aistudio.google.com/app/apikey) para gerar sua chave.
2.  **IMPORTANTE:** Nunca cole sua chave diretamente no código. Use o gerenciador de "Secrets" do Colab.
3.  Clique no ícone de **chave** (🔑) na barra lateral esquerda.
4.  Adicione um novo segredo chamado `GOOGLE_API_KEY` e cole sua chave lá.

In [2]:
import google.generativeai as genai
from google.colab import userdata
import os

try:
    # Tenta buscar a API Key dos Secrets do Colab
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua API Key na aba 'Secrets' (ícone de chave) à esquerda e nomeie-a 'GOOGLE_API_KEY'.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")
    print("Certifique-se de que sua API Key é válida e foi adicionada corretamente.")

API Key configurada com sucesso!


## Passo 3: Exemplo 1 - Geração de Texto Simples

Este é o caso de uso mais básico: enviar um prompt e receber uma resposta.

In [3]:
# Configura o modelo
# Vamos usar o gemini-2.5-flash-preview-09-2025, que é rápido e eficiente
model = genai.GenerativeModel('gemini-2.5-flash-preview-09-2025')

# Envia um prompt
prompt = "Escreva um poema curto sobre o Brasil, em 4 linhas."
print(f"Enviando prompt: {prompt}\n")

response = model.generate_content(prompt)

# Imprime a resposta
try:
    print("--- Resposta do Gemini ---")
    print(response.text)
except ValueError as e:
    print(f"Erro ao gerar conteúdo. O modelo pode ter bloqueado a resposta. Detalhes: {e}")
    print(f"Resposta completa: {response}")

Enviando prompt: Escreva um poema curto sobre o Brasil, em 4 linhas.

--- Resposta do Gemini ---
Verde e amarelo, luz que aquece,
O samba corre e nunca arrefece.
A floresta guarda o mar e o sol,
Brasil, eterno e vasto farol.


## Passo 4: Exemplo 2 - Chat (Conversa com Histórico)

Para casos de uso como chatbots, você pode criar um objeto de chat que mantém o histórico da conversa.

In [4]:
# O modelo de chat mantém o histórico
chat_model = genai.GenerativeModel('gemini-2.5-flash-preview-09-2025')
chat = chat_model.start_chat(history=[])

# Pergunta 1
prompt1 = "Qual é a capital da França?"
print(f"Usuário: {prompt1}\n")

response1 = chat.send_message(prompt1)
print(f"Gemini: {response1.text}\n")

# Pergunta 2 (com contexto)
prompt2 = "E qual é o principal ponto turístico de lá?"
print(f"Usuário: {prompt2}\n")

response2 = chat.send_message(prompt2)
print(f"Gemini: {response2.text}\n")

# Opcional: ver o histórico
print("\n--- Histórico da Conversa Armazenado ---")
for message in chat.history:
    # Exibindo apenas as partes de texto
    print(f"**{message.role}**: {message.parts[0].text}")

Usuário: Qual é a capital da França?

Gemini: A capital da França é **Paris**.

Usuário: E qual é o principal ponto turístico de lá?

Gemini: O principal e mais famoso ponto turístico de Paris, sendo o seu símbolo máximo, é a **Torre Eiffel** (La Tour Eiffel).

No entanto, o **Museu do Louvre** também é um ponto extremamente importante e mundialmente famoso, conhecido por abrigar obras como a Mona Lisa e a Vênus de Milo.


--- Histórico da Conversa Armazenado ---
**user**: Qual é a capital da França?
**model**: A capital da França é **Paris**.
**user**: E qual é o principal ponto turístico de lá?
**model**: O principal e mais famoso ponto turístico de Paris, sendo o seu símbolo máximo, é a **Torre Eiffel** (La Tour Eiffel).

No entanto, o **Museu do Louvre** também é um ponto extremamente importante e mundialmente famoso, conhecido por abrigar obras como a Mona Lisa e a Vênus de Milo.


## Passo 5: Exemplo 3 - Multimodal (Texto e Imagem)

O Gemini pode receber entradas que misturam texto e imagens.

In [8]:
# Primeiro, vamos baixar uma imagem de exemplo
!wget -q -O pao_de_acucar.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Vista_do_Morro_Dona_Marta.jpg/1920px-Vista_do_Morro_Dona_Marta.jpg
print("Imagem 'pao_de_acucar.jpg' baixada.")

Imagem 'pao_de_acucar.jpg' baixada.


In [9]:
import PIL.Image

try:
    img = PIL.Image.open('pao_de_acucar.jpg')

    # Para multimodalidade, usamos o mesmo modelo flash
    multimodal_model = genai.GenerativeModel('gemini-2.5-flash-preview-09-2025')

    # Envia o prompt com texto e imagem
    prompt_multimodal = "Que lugar é este e em que cidade ele fica?"
    print(f"Enviando prompt: {prompt_multimodal} + [IMAGEM]\n")

    response_multimodal = multimodal_model.generate_content([prompt_multimodal, img])

    print("--- Resposta do Gemini ---")
    print(response_multimodal.text)

except FileNotFoundError:
    print("Erro: Arquivo 'pao_de_acucar.jpg' não encontrado. Tente executar a célula anterior novamente.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Enviando prompt: Que lugar é este e em que cidade ele fica? + [IMAGEM]

--- Resposta do Gemini ---
Este lugar icônico é a **Baía de Guanabara**, com destaque para o famoso **Pão de Açúcar** e o **Morro da Urca** em primeiro plano.

A cidade onde ele se localiza é:

**Rio de Janeiro, Brasil.**

A foto foi tirada provavelmente de um ponto alto na zona sul ou central da cidade, como o **Corcovado** ou o **Mirante Dona Marta**, oferecendo uma vista panorâmica que engloba a Enseada de Botafogo (parcialmente visível na esquerda) e a entrada da Baía de Guanabara.


## Fim!

Você aprendeu a usar a API do Gemini para as tarefas mais comuns. Sinta-se à vontade para modificar os prompts e testar com suas próprias imagens.